## 1. Setup and Imports

In [ ]:
# Install required packages if not already installed
# !pip install pandas numpy matplotlib seaborn cvxpy
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime, date, timedelta
from typing import Dict, List, Tuple, Optional
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✓ Libraries imported successfully")

## 2. Mock Data Generation

This section generates realistic mock data that matches the exact database format used by the optimization algorithm.

In [ ]:
class MockDataGenerator:
    """Generates mock data for backtesting the optimization algorithm."""
    
    def __init__(self, seed=42):
        """Initialize the mock data generator with a random seed."""
        np.random.seed(seed)
        self.date = date.today()
        
    def generate_pools(self, num_pools=20):
        """Generate mock pool data matching the database format.
        
        Returns:
            pd.DataFrame: Pool data with columns matching pool_daily_metrics and pools tables
        """
        # Define common stablecoin pools
        pool_templates = [
            {
                'symbol': 'USDC-USDT',
                'chain': 'Ethereum',
                'protocol': 'Curve',
                'underlying_tokens': ['USDC', 'USDT']
            },
            {
                'symbol': 'DAI-USDC',
                'chain': 'Ethereum',
                'protocol': 'Curve',
                'underlying_tokens': ['DAI', 'USDC']
            },
            {
                'symbol': 'USDC-DAI-USDT',
                'chain': 'Ethereum',
                'protocol': 'Curve',
                'underlying_tokens': ['USDC', 'DAI', 'USDT']
            },
            {
                'symbol': 'USDC',
                'chain': 'Ethereum',
                'protocol': 'Aave',
                'underlying_tokens': ['USDC']
            },
            {
                'symbol': 'DAI',
                'chain': 'Ethereum',
                'protocol': 'Aave',
                'underlying_tokens': ['DAI']
            },
            {
                'symbol': 'USDT',
                'chain': 'Ethereum',
                'protocol': 'Aave',
                'underlying_tokens': ['USDT']
            },
            {
                'symbol': 'USDC-USDT',
                'chain': 'Polygon',
                'protocol': 'Curve',
                'underlying_tokens': ['USDC', 'USDT']
            },
            {
                'symbol': 'USDC',
                'chain': 'Polygon',
                'protocol': 'Aave',
                'underlying_tokens': ['USDC']
            }
        ]
        
        pools = []
        for i in range(num_pools):
            template = pool_templates[i % len(pool_templates)]
            pool_id = f"pool_{i+1:03d}"
            
            # Generate realistic APY values (1-15%)
            base_apy = np.random.uniform(0.01, 0.15)
            
            # Add some variation based on protocol and chain
            if template['protocol'] == 'Aave':
                base_apy *= 0.9  # Aave typically lower APY
            elif template['protocol'] == 'Curve' and len(template['underlying_tokens']) == 2:
                base_apy *= 1.1  # 2-pool Curve typically higher APY
            
            # Generate realistic TVL values ($100K - $100M)
            base_tvl = np.random.uniform(100000, 100000000)
            
            pools.append({
                'pool_id': pool_id,
                'symbol': template['symbol'],
                'chain': template['chain'],
                'protocol': template['protocol'],
                'underlying_tokens': json.dumps(template['underlying_tokens']),
                'forecasted_apy': base_apy * 100,  # Convert to percentage
                'forecasted_tvl': base_tvl,
                'is_filtered_out': False,
                'is_active': True
            })
        
        return pd.DataFrame(pools)
    
    def generate_token_prices(self, tokens):
        """Generate mock token prices matching the database format.
        
        Args:
            tokens: List of token symbols
            
        Returns:
            Dict: Token prices in USD
        """
        prices = {}
        for token in tokens:
            if token in ['USDC', 'USDT', 'DAI', 'FRAX', 'TUSD', 'BUSD', 'GUSD', 'USDP']:
                # Stablecoins - very close to $1 with small variations
                prices[token] = 1.0 + np.random.normal(0, 0.002)
            elif token == 'ETH':
                # ETH price around $3000 with more volatility
                prices[token] = 3000 + np.random.normal(0, 100)
            elif token == 'WBTC':
                # WBTC price around $60000
                prices[token] = 60000 + np.random.normal(0, 2000)
            else:
                # Other tokens - reasonable price range
                prices[token] = np.random.uniform(0.5, 100)
        
        return prices
    
    def generate_balances(self, tokens, total_aum=1000000):
        """Generate mock wallet and allocation balances.
        
        Args:
            tokens: List of token symbols
            total_aum: Total assets under management in USD
            
        Returns:
            Tuple of (warm_wallet_balances, current_allocations)
        """
        # Generate warm wallet balances (20-40% of total AUM)
        warm_wallet_pct = np.random.uniform(0.2, 0.4)
        warm_wallet_usd = total_aum * warm_wallet_pct
        
        warm_wallet = {}
        remaining_usd = warm_wallet_usd
        
        # Distribute among stablecoins
        stablecoins = [t for t in tokens if t in ['USDC', 'USDT', 'DAI']]
        if stablecoins:
            for i, token in enumerate(stablecoins):
                if i == len(stablecoins) - 1:
                    # Last token gets remaining amount
                    amount_usd = remaining_usd
                else:
                    # Random distribution
                    amount_usd = remaining_usd * np.random.uniform(0.2, 0.5)
                    remaining_usd -= amount_usd
                
                price = 1.0  # Stablecoins are ~$1
                warm_wallet[token] = amount_usd / price
        
        # Generate current allocations (60-80% of total AUM)
        allocated_usd = total_aum - warm_wallet_usd
        current_allocations = {}
        
        # Create some existing allocations
        num_allocations = np.random.randint(3, 8)
        pool_ids = [f"pool_{i+1:03d}" for i in range(num_allocations)]
        
        remaining_allocated_usd = allocated_usd
        for i, pool_id in enumerate(pool_ids):
            if i == len(pool_ids) - 1:
                amount_usd = remaining_allocated_usd
            else:
                amount_usd = remaining_allocated_usd * np.random.uniform(0.1, 0.3)
                remaining_allocated_usd -= amount_usd
            
            # Randomly assign to tokens
            token = np.random.choice(stablecoins if stablecoins else ['USDC'])
            price = 1.0
            amount = amount_usd / price
            
            current_allocations[(pool_id, token)] = amount
        
        return warm_wallet, current_allocations
    
    def generate_gas_fees(self):
        """Generate mock gas fee data.
        
        Returns:
            Tuple: (eth_price_usd, base_fee_transfer_gwei, base_fee_swap_gwei, priority_fee_gwei, min_gas_units)
        """
        eth_price = 3000 + np.random.normal(0, 100)
        base_fee_transfer_gwei = np.random.uniform(5, 15)
        base_fee_swap_gwei = np.random.uniform(20, 40)
        priority_fee_gwei = np.random.uniform(5, 15)
        min_gas_units = 21000
        
        return eth_price, base_fee_transfer_gwei, base_fee_swap_gwei, priority_fee_gwei, min_gas_units
    
    def generate_allocation_parameters(self, custom_overrides=None):
        """Generate mock allocation parameters.
        
        Args:
            custom_overrides: Optional dictionary of parameter overrides
            
        Returns:
            Dict: Allocation parameters
        """
        params = {
            'run_id': f"mock_run_{int(time.time())}",
            'timestamp': datetime.now(),
            'max_alloc_percentage': 0.25,  # 25% max per pool
            'conversion_rate': 0.0004,     # 0.04% conversion fee
            'min_transaction_value': 50.0,  # $50 minimum transaction
            'tvl_limit_percentage': 0.05,  # 5% of pool TVL max
            'min_pools': 5,                # Minimum 5 pools
            
            # Filtering parameters
            'token_marketcap_limit': 1000000000.0,
            'pool_tvl_limit': 100000.0,
            'pool_apy_limit': 0.01,
            'pool_pair_tvl_ratio_min': 0.3,
            'pool_pair_tvl_ratio_max': 0.5,
            
            # Group allocation limits
            'group1_max_pct': 0.35,
            'group2_max_pct': 0.35,
            'group3_max_pct': 0.3,
            
            # Position limits
            'position_max_pct_total_assets': 0.25,
            'position_max_pct_pool_tvl': 0.05,
            
            # APY and volatility limits
            'group1_apy_delta_max': 0.01,
            'group1_7d_stddev_max': 0.015,
            'group1_30d_stddev_max': 0.02,
            'group2_apy_delta_max': 0.03,
            'group2_7d_stddev_max': 0.04,
            'group2_30d_stddev_max': 0.05,
            'group3_apy_delta_min': 0.03,
            'group3_7d_stddev_min': 0.04,
            'group3_30d_stddev_min': 0.02,
            
            # Icebox parameters
            'icebox_ohlc_l_threshold_pct': 0.02,
            'icebox_ohlc_l_days_threshold': 2,
            'icebox_ohlc_c_threshold_pct': 0.01,
            'icebox_ohlc_c_days_threshold': 1,
            'icebox_recovery_l_days_threshold': 2,
            'icebox_recovery_c_days_threshold': 3
        }
        
        # Apply custom overrides if provided
        if custom_overrides:
            params.update(custom_overrides)
        
        return params

print("✓ MockDataGenerator class defined")

## 3. Generate Mock Data

In [ ]:
# Initialize mock data generator
generator = MockDataGenerator(seed=42)

# Generate pools
pools_df = generator.generate_pools(num_pools=15)
print(f"Generated {len(pools_df)} pools")
print("\nSample pools:")
display(pools_df.head())

# Extract tokens from pools
tokens = set()
for _, row in pools_df.iterrows():
    underlying_tokens = json.loads(row['underlying_tokens'])
    tokens.update(underlying_tokens)
tokens = list(tokens)
tokens.append('ETH')  # Add ETH for gas fees

print(f"\nTokens in universe: {tokens}")

# Generate token prices
token_prices = generator.generate_token_prices(tokens)
print("\nToken prices:")
for token, price in token_prices.items():
    print(f"  {token}: ${price:.4f}")

# Generate balances
warm_wallet, current_allocations = generator.generate_balances(tokens, total_aum=1000000)
print("\nWarm wallet balances:")
for token, amount in warm_wallet.items():
    print(f"  {token}: {amount:,.2f}")

print("\nCurrent allocations:")
for (pool_id, token), amount in current_allocations.items():
    print(f"  {pool_id} - {token}: {amount:,.2f}")

# Generate gas fees
eth_price, base_fee_transfer, base_fee_swap, priority_fee, min_gas = generator.generate_gas_fees()
print(f"\nGas fee data:")
print(f"  ETH price: ${eth_price:.2f}")
print(f"  Base fee (transfer): {base_fee_transfer:.2f} Gwei")
print(f"  Base fee (swap): {base_fee_swap:.2f} Gwei")
print(f"  Priority fee: {priority_fee:.2f} Gwei")
print(f"  Min gas units: {min_gas}")

# Generate allocation parameters
alloc_params = generator.generate_allocation_parameters()
print("\nKey allocation parameters:")
print(f"  Max allocation per pool: {alloc_params['max_alloc_percentage']:.1%}")
print(f"  TVL limit percentage: {alloc_params['tvl_limit_percentage']:.1%}")
print(f"  Conversion rate: {alloc_params['conversion_rate']:.4%}")
print(f"  Minimum pools: {alloc_params['min_pools']}")

## 4. Optimization Implementation

This section includes the optimization algorithm adapted to work with mock data instead of database connections.

In [ ]:
# Import optimization classes from the actual implementation
import sys
import os

# Add project root to Python path
sys.path.append(os.getcwd())

try:
    from asset_allocation.optimize_allocations import AllocationOptimizer, calculate_aum, build_token_universe
    print("✓ Successfully imported optimization classes")
except ImportError as e:
    print(f"✗ Failed to import optimization classes: {e}")
    print("Please ensure you're running this from the project root directory")
    print("The asset_allocation module should be available")

In [ ]:
# Helper functions for gas fee calculation (from the original implementation)
def calculate_gas_fee_usd(gas_units, base_fee_gwei, priority_fee_gwei, eth_price_usd):
    """Calculate gas fee in USD."""
    total_fee_gwei = gas_units * (base_fee_gwei + priority_fee_gwei)
    gas_fee_usd = total_fee_gwei * 1e-9 * eth_price_usd
    return gas_fee_usd

def calculate_transaction_gas_fees(eth_price_usd, base_fee_transfer_gwei, 
                                   base_fee_swap_gwei, priority_fee_gwei, 
                                   min_gas_units):
    """Calculate gas fees for different transaction types."""
    # Pool allocation/withdrawal gas fee
    pool_transaction_gas_fee_usd = calculate_gas_fee_usd(
        min_gas_units, base_fee_transfer_gwei, priority_fee_gwei, eth_price_usd
    )
    
    # Token swap gas fee
    token_swap_gas_fee_usd = calculate_gas_fee_usd(
        min_gas_units, base_fee_swap_gwei, priority_fee_gwei, eth_price_usd
    )
    
    gas_fees = {
        'allocation': pool_transaction_gas_fee_usd,
        'withdrawal': pool_transaction_gas_fee_usd,
        'conversion': token_swap_gas_fee_usd,
        'transfer': pool_transaction_gas_fee_usd,
        'deposit': pool_transaction_gas_fee_usd
    }
    
    return gas_fees

print("✓ Helper functions defined")

## 5. Run Optimization with Mock Data

In [ ]:
# Calculate gas fees
gas_fees = calculate_transaction_gas_fees(
    eth_price, base_fee_transfer, base_fee_swap, priority_fee, min_gas
)

print("Gas fees by transaction type:")
for txn_type, fee in gas_fees.items():
    print(f"  {txn_type}: ${fee:.6f}")

# Calculate total AUM
total_aum = calculate_aum(warm_wallet, current_allocations, token_prices)
print(f"\nTotal AUM: ${total_aum:,.2f}")

# Build token universe
token_universe = build_token_universe(pools_df, warm_wallet, current_allocations)
print(f"Token universe: {token_universe}")

# Initialize optimizer
print("\nInitializing optimizer...")
optimizer = AllocationOptimizer(
    pools_df=pools_df,
    token_prices=token_prices,
    warm_wallet=warm_wallet,
    current_allocations=current_allocations,
    gas_fees=gas_fees,
    alloc_params=alloc_params
)

print("✓ Optimizer initialized successfully")

In [ ]:
total_aum

In [ ]:
# Solve optimization problem
print("Solving optimization problem...")
start_time = time.time()

try:
    success = optimizer.solve(verbose=True)
    solve_time = time.time() - start_time
    
    if success:
        print(f"✓ Optimization solved successfully in {solve_time:.3f} seconds")
    else:
        print("✗ Optimization failed")
        
except Exception as e:
    print(f"✗ Error during optimization: {e}")
    import traceback
    traceback.print_exc()

## 6. Analyze Results

In [ ]:
if success:
    # Extract and format results
    formatted_results = optimizer.format_results()
    
    print("=" * 80)
    print("OPTIMIZATION RESULTS")
    print("=" * 80)
    
    # Display final allocations
    print("\nFINAL ALLOCATIONS:")
    total_allocated = 0
    for pool_id, pool_data in formatted_results["final_allocations"].items():
        pool_total_usd = sum(token_data['amount_usd'] for token_data in pool_data["tokens"].values())
        pool_percentage = pool_total_usd / optimizer.total_aum
        total_allocated += pool_total_usd
        print(f"\nPool: {pool_id} ({pool_data['pool_symbol']}) - Total: ${pool_total_usd:,.2f} ({pool_percentage:.2%})")
        for token, token_data in pool_data["tokens"].items():
            print(f"  {token}: {token_data['amount']:,.2f} (${token_data['amount_usd']:,.2f})")
    
    # Display unallocated tokens
    print("\nUNALLOCATED TOKENS (in warm wallet):")
    total_unallocated = 0
    for token, token_data in formatted_results["unallocated_tokens"].items():
        total_unallocated += token_data['amount_usd']
        print(f"  {token}: {token_data['amount']:,.2f} (${token_data['amount_usd']:,.2f})")
    
    # Display transaction summary
    print("\nTRANSACTION SUMMARY:")
    transactions = formatted_results["transactions"]
    total_gas_cost = sum(txn.get('gas_cost_usd', 0) for txn in transactions)
    total_conversion_cost = sum(txn.get('conversion_cost_usd', 0) for txn in transactions)
    total_cost = sum(txn.get('total_cost_usd', 0) for txn in transactions)
    
    print(f"  Total transactions: {len(transactions)}")
    print(f"  Total gas cost: ${total_gas_cost:.4f}")
    print(f"  Total conversion cost: ${total_conversion_cost:.4f}")
    print(f"  Total transaction cost: ${total_cost:.4f}")
    
    # Display allocation summary
    print("\nALLOCATION SUMMARY:")
    print(f"  Total AUM: ${optimizer.total_aum:,.2f}")
    print(f"  Total allocated: ${total_allocated:,.2f} ({total_allocated/optimizer.total_aum:.2%})")
    print(f"  Total unallocated: ${total_unallocated:,.2f} ({total_unallocated/optimizer.total_aum:.2%})")
    print(f"  Transaction costs: ${total_cost:.4f} ({total_cost/optimizer.total_aum:.4%})")
    print(f"  Net allocated: ${total_allocated + total_unallocated + total_cost:,.2f}")
    
else:
    print("No results to display - optimization failed")

## 7. Visualization

In [ ]:
if success:
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Optimization Results Analysis', fontsize=16, fontweight='bold')
    
    # 1. Allocation by pool
    pool_data = []
    for pool_id, pool_info in formatted_results["final_allocations"].items():
        pool_total = sum(token_data['amount_usd'] for token_data in pool_info["tokens"].values())
        pool_data.append({
            'pool': f"{pool_id}\n({pool_info['pool_symbol']})",
            'amount_usd': pool_total,
            'percentage': pool_total / optimizer.total_aum * 100
        })
    
    pool_df = pd.DataFrame(pool_data)
    pool_df = pool_df.sort_values('amount_usd', ascending=False)
    
    axes[0, 0].bar(range(len(pool_df)), pool_df['amount_usd'])
    axes[0, 0].set_title('Allocation by Pool')
    axes[0, 0].set_xlabel('Pool')
    axes[0, 0].set_ylabel('Amount (USD)')
    axes[0, 0].set_xticks(range(len(pool_df)))
    axes[0, 0].set_xticklabels(pool_df['pool'], rotation=45, ha='right')
    
    # Add percentage labels on bars
    for i, row in pool_df.iterrows():
        axes[0, 0].text(i, row['amount_usd'], f"{row['percentage']:.1f}%", 
                       ha='center', va='bottom')
    
    # 2. Token distribution
    token_dist = {}
    for pool_id, pool_info in formatted_results["final_allocations"].items():
        for token, token_data in pool_info["tokens"].items():
            if token not in token_dist:
                token_dist[token] = 0
            token_dist[token] += token_data['amount_usd']
    
    # Add unallocated tokens
    for token, token_data in formatted_results["unallocated_tokens"].items():
        if token not in token_dist:
            token_dist[token] = 0
        token_dist[token] += token_data['amount_usd']
    
    tokens_df = pd.DataFrame(list(token_dist.items()), columns=['token', 'amount_usd'])
    tokens_df = tokens_df.sort_values('amount_usd', ascending=False)
    
    axes[0, 1].pie(tokens_df['amount_usd'], labels=tokens_df['token'], autopct='%1.1f%%')
    axes[0, 1].set_title('Token Distribution')
    
    # 3. APY distribution
    apy_data = []
    for pool_id, pool_info in formatted_results["final_allocations"].items():
        pool_total = sum(token_data['amount_usd'] for token_data in pool_info["tokens"].values())
        pool_row = pools_df[pools_df['pool_id'] == pool_id].iloc[0]
        apy_data.append({
            'pool': pool_info['pool_symbol'],
            'apy': pool_row['forecasted_apy'],
            'amount_usd': pool_total
        })
    
    apy_df = pd.DataFrame(apy_data)
    
    axes[1, 0].scatter(apy_df['apy'], apy_df['amount_usd'], s=100, alpha=0.7)
    axes[1, 0].set_title('Allocation vs APY')
    axes[1, 0].set_xlabel('APY (%)')
    axes[1, 0].set_ylabel('Amount (USD)')
    
    # Add pool labels
    for _, row in apy_df.iterrows():
        axes[1, 0].annotate(row['pool'], (row['apy'], row['amount_usd']), 
                           xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    # 4. Transaction cost breakdown
    txn_types = ['WITHDRAWAL', 'CONVERSION', 'ALLOCATION']
    txn_costs = []
    
    for txn_type in txn_types:
        type_txns = [t for t in transactions if t['type'] == txn_type]
        total_cost = sum(t.get('total_cost_usd', 0) for t in type_txns)
        txn_costs.append(total_cost)
    
    axes[1, 1].bar(txn_types, txn_costs)
    axes[1, 1].set_title('Transaction Costs by Type')
    axes[1, 1].set_xlabel('Transaction Type')
    axes[1, 1].set_ylabel('Cost (USD)')
    
    # Add cost labels on bars
    for i, cost in enumerate(txn_costs):
        axes[1, 1].text(i, cost, f"${cost:.4f}", ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("No visualizations available - optimization failed")

## 8. Backtesting Scenarios

This section provides different scenarios to test the optimization algorithm under various conditions.

In [ ]:
def run_scenario(scenario_name, custom_overrides=None, custom_pool_params=None, existing_pools_df=None):
    """Run a backtesting scenario with custom parameters.
    
    Args:
        scenario_name: Name of scenario
        custom_overrides: Custom allocation parameter overrides
        custom_pool_params: Custom pool generation parameters
        existing_pools_df: Existing pools DataFrame to use (if None, generate new)
        
    Returns:
        Dict: Scenario results
    """
    print(f"\n{'='*80}")
    print(f"RUNNING SCENARIO: {scenario_name}")
    print(f"{'='*80}")
    
    # Always create a generator instance
    generator = MockDataGenerator(seed=42)
    
    # Generate custom data if provided
    if custom_pool_params:
        pools_df = generator.generate_pools(**custom_pool_params)
        
        # Extract tokens
        tokens = set()
        for _, row in pools_df.iterrows():
            underlying_tokens = json.loads(row['underlying_tokens'])
            tokens.update(underlying_tokens)
        tokens = list(tokens)
        tokens.append('ETH')
        
        # Generate other data
        token_prices = generator.generate_token_prices(tokens)
        warm_wallet, current_allocations = generator.generate_balances(tokens)
    else:
        # Use existing data
        pools_df = existing_pools_df if existing_pools_df is not None else pools_df
        tokens = list(set([t for sublist in [json.loads(ut) for ut in pools_df['underlying_tokens']] for t in sublist]))
        tokens.append('ETH')
        
        # Use existing prices and balances if available, otherwise generate
        if 'token_prices' in globals():
            token_prices = globals()['token_prices']
            warm_wallet = globals()['warm_wallet']
            current_allocations = globals()['current_allocations']
        else:
            token_prices = generator.generate_token_prices(tokens)
            warm_wallet, current_allocations = generator.generate_balances(tokens)
    
    # Generate parameters with overrides
    alloc_params = generator.generate_allocation_parameters(custom_overrides)
    
    # Initialize optimizer
    optimizer = AllocationOptimizer(
        pools_df=pools_df,
        token_prices=token_prices,
        warm_wallet=warm_wallet,
        current_allocations=current_allocations,
        gas_fees=gas_fees,
        alloc_params=alloc_params
    )
    
    # Solve optimization
    start_time = time.time()
    success = optimizer.solve(verbose=False)
    solve_time = time.time() - start_time
    
    if not success:
        return {
            'scenario': scenario_name,
            'success': False,
            'solve_time': solve_time,
            'error': 'Optimization failed'
        }
    
    # Extract results
    results = optimizer.format_results()
    
    # Calculate metrics
    total_allocated = sum(
        sum(token_data['amount_usd'] for token_data in pool_data["tokens"].values())
        for pool_data in results["final_allocations"].values()
    )
    
    total_unallocated = sum(
        token_data['amount_usd'] for token_data in results["unallocated_tokens"].values()
    )
    
    total_cost = sum(txn.get('total_cost_usd', 0) for txn in results["transactions"])
    
    # Calculate weighted average APY
    weighted_apy = 0
    for pool_id, pool_data in results["final_allocations"].items():
        pool_total = sum(token_data['amount_usd'] for token_data in pool_data["tokens"].values())
        pool_row = pools_df[pools_df['pool_id'] == pool_id].iloc[0]
        pool_apy = pool_row['forecasted_apy']
        weighted_apy += pool_total * pool_apy
    
    if total_allocated > 0:
        weighted_apy /= total_allocated
    
    return {
        'scenario': scenario_name,
        'success': True,
        'solve_time': solve_time,
        'total_aum': optimizer.total_aum,
        'total_allocated': total_allocated,
        'total_unallocated': total_unallocated,
        'total_cost': total_cost,
        'num_pools': len(results["final_allocations"]),
        'num_transactions': len(results["transactions"]),
        'weighted_apy': weighted_apy,
        'allocation_pct': total_allocated / optimizer.total_aum,
        'cost_pct': total_cost / optimizer.total_aum,
        'results': results
    }

print("✓ Scenario runner function defined")

In [ ]:
# Define backtesting scenarios
scenarios = [
    {
        'name': 'Baseline',
        'overrides': {},
        'pool_params': None
    },
    {
        'name': 'High Max Allocation',
        'overrides': {'max_alloc_percentage': 0.5},  # 50% max per pool
        'pool_params': None
    },
    {
        'name': 'Low TVL Limit',
        'overrides': {'tvl_limit_percentage': 0.01},  # 1% of pool TVL
        'pool_params': None
    },
    {
        'name': 'High Conversion Cost',
        'overrides': {'conversion_rate': 0.001},  # 0.1% conversion fee
        'pool_params': None
    },
    {
        'name': 'Many Small Pools',
        'overrides': {},
        'pool_params': {'num_pools': 30}
    },
    {
        'name': 'Few Large Pools',
        'overrides': {},
        'pool_params': {'num_pools': 5}
    }
]

# Run all scenarios
scenario_results = []

for scenario in scenarios:
    result = run_scenario(
        scenario['name'],
        scenario['overrides'],
        scenario['pool_params'],
        pools_df  # Pass the existing pools_df for scenarios that don't generate new pools
    )
    scenario_results.append(result)
    
    if result['success']:
        print(f"✓ {scenario['name']} completed successfully")
        print(f"  Solve time: {result['solve_time']:.3f}s")
        print(f"  Allocation: {result['allocation_pct']:.2%}")
        print(f"  Weighted APY: {result['weighted_apy']:.2f}%")
        print(f"  Cost: {result['cost_pct']:.4%}")
    else:
        print(f"✗ {scenario['name']} failed: {result.get('error', 'Unknown error')}")

In [ ]:
# Compare scenarios
successful_results = [r for r in scenario_results if r['success']]

if successful_results:
    comparison_df = pd.DataFrame(successful_results)
    
    # Create comparison visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Scenario Comparison', fontsize=16, fontweight='bold')
    
    # 1. Allocation percentage
    axes[0, 0].bar(comparison_df['scenario'], comparison_df['allocation_pct'])
    axes[0, 0].set_title('Allocation Percentage')
    axes[0, 0].set_ylabel('Percentage of AUM')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Add percentage labels
    for i, pct in enumerate(comparison_df['allocation_pct']):
        axes[0, 0].text(i, pct, f"{pct:.1%}", ha='center', va='bottom')
    
    # 2. Weighted APY
    axes[0, 1].bar(comparison_df['scenario'], comparison_df['weighted_apy'])
    axes[0, 1].set_title('Weighted Average APY')
    axes[0, 1].set_ylabel('APY (%)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Add APY labels
    for i, apy in enumerate(comparison_df['weighted_apy']):
        axes[0, 1].text(i, apy, f"{apy:.2f}%", ha='center', va='bottom')
    
    # 3. Cost percentage
    axes[1, 0].bar(comparison_df['scenario'], comparison_df['cost_pct'] * 100)
    axes[1, 0].set_title('Transaction Cost Percentage')
    axes[1, 0].set_ylabel('Cost (% of AUM)')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Add cost labels
    for i, cost in enumerate(comparison_df['cost_pct']):
        axes[1, 0].text(i, cost * 100, f"{cost*100:.3f}%", ha='center', va='bottom')
    
    # 4. Solve time
    axes[1, 1].bar(comparison_df['scenario'], comparison_df['solve_time'])
    axes[1, 1].set_title('Solve Time')
    axes[1, 1].set_ylabel('Time (seconds)')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    # Add time labels
    for i, time_val in enumerate(comparison_df['solve_time']):
        axes[1, 1].text(i, time_val, f"{time_val:.3f}s", ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Display comparison table
    print("\nSCENARIO COMPARISON TABLE:")
    display_cols = ['scenario', 'allocation_pct', 'weighted_apy', 'cost_pct', 'solve_time', 'num_pools']
    display_df = comparison_df[display_cols].copy()
    display_df['allocation_pct'] = display_df['allocation_pct'].apply(lambda x: f"{x:.2%}")
    display_df['cost_pct'] = display_df['cost_pct'].apply(lambda x: f"{x:.4%}")
    display_df['weighted_apy'] = display_df['weighted_apy'].apply(lambda x: f"{x:.2f}%")
    display_df['solve_time'] = display_df['solve_time'].apply(lambda x: f"{x:.3f}s")
    
    display(display_df)
    
else:
    print("No successful scenarios to compare")

## 9. Custom Scenario Testing

Use this section to test your own custom scenarios and parameter configurations.

In [ ]:
# Example: Test a custom scenario
# Modify these parameters to test your own scenarios

custom_overrides = {
    'max_alloc_percentage': 0.3,      # 30% max per pool
    'tvl_limit_percentage': 0.03,     # 3% of pool TVL
    'conversion_rate': 0.0005,        # 0.05% conversion fee
    'min_transaction_value': 100.0    # $100 minimum transaction
}

custom_pool_params = {
    'num_pools': 25  # Generate 25 pools
}

# Run custom scenario
custom_result = run_scenario(
    'Custom Scenario',
    custom_overrides,
    custom_pool_params
)

if custom_result['success']:
    print(f"✓ Custom scenario completed successfully")
    print(f"  Total AUM: ${custom_result['total_aum']:,.2f}")
    print(f"  Allocation: {custom_result['allocation_pct']:.2%}")
    print(f"  Weighted APY: {custom_result['weighted_apy']:.2f}%")
    print(f"  Cost: ${custom_result['total_cost']:.4f} ({custom_result['cost_pct']:.4%})")
    print(f"  Pools used: {custom_result['num_pools']}")
    print(f"  Transactions: {custom_result['num_transactions']}")
else:
    print(f"✗ Custom scenario failed: {custom_result.get('error', 'Unknown error')}")

## 10. Export Results

Export backtesting results for further analysis.

In [ ]:
# Export scenario results to CSV
if successful_results:
    results_df = pd.DataFrame(successful_results)
    
    # Select columns for export
    export_cols = [
        'scenario', 'total_aum', 'total_allocated', 'total_unallocated',
        'total_cost', 'num_pools', 'num_transactions', 'weighted_apy',
        'allocation_pct', 'cost_pct', 'solve_time'
    ]
    
    export_df = results_df[export_cols].copy()
    
    # Save to CSV
    export_df.to_csv('backtesting_results.csv', index=False)
    print("✓ Results exported to backtesting_results.csv")
    
    # Display export preview
    print("\nExport preview:")
    display(export_df.head())
    
    # Save detailed results for each scenario
    for result in successful_results:
        scenario_name = result['scenario'].replace(' ', '_').lower()
        
        # Save allocations
        if result['results']['final_allocations']:
            allocations_data = []
            for pool_id, pool_data in result['results']['final_allocations'].items():
                for token, token_data in pool_data['tokens'].items():
                    allocations_data.append({
                        'scenario': result['scenario'],
                        'pool_id': pool_id,
                        'pool_symbol': pool_data['pool_symbol'],
                        'token': token,
                        'amount': token_data['amount'],
                        'amount_usd': token_data['amount_usd']
                    })
            
            alloc_df = pd.DataFrame(allocations_data)
            alloc_df.to_csv(f'allocations_{scenario_name}.csv', index=False)
        
        # Save transactions
        if result['results']['transactions']:
            txn_df = pd.DataFrame(result['results']['transactions'])
            txn_df['scenario'] = result['scenario']
            txn_df.to_csv(f'transactions_{scenario_name}.csv', index=False)
    
    print("✓ Detailed allocations and transactions exported for each scenario")
    
else:
    print("No results to export")

## 11. Next Steps

### For Data Scientists:

1. **Modify Mock Data**: Adjust `MockDataGenerator` class to create more realistic scenarios based on your market knowledge

2. **Add Custom Scenarios**: Create your own scenarios in section 9 to test specific market conditions or parameter configurations

3. **Integrate Real Data**: Replace mock data with real historical data by connecting to your database

4. **Extended Analysis**: Add more sophisticated analysis like:
   - Monte Carlo simulations
   - Sensitivity analysis
   - Risk metrics calculation
   - Performance attribution

5. **Parameter Optimization**: Use this framework to find optimal parameter settings for different market regimes

### Integration with Production:

To integrate this with the production system:

1. Replace mock data generators with actual database queries from `asset_allocation/optimize_allocations.py`

2. Use the same parameter structure as the production system

3. Implement the same result storage mechanism

4. Add monitoring and alerting for optimization performance

### Key Files to Reference:

- `asset_allocation/optimize_allocations.py`: Main optimization implementation
- `asset_allocation/data_quality_report.py`: Data quality validation
- `database/schema/`: Database schema definitions
- `config.py`: Configuration parameters

### Contact Information:

For questions about the optimization algorithm or integration, refer to the project documentation or contact the development team.

# Backtesting on Real Data

In [ ]:
from database.db_utils import get_db_connection

In [ ]:
# ./cloud-sql-proxy innate-concept-430315-q2:us-central1:defiyieldopt-db-instance
engine = get_db_connection()

In [ ]:
pools_metadata_df = pd.read_sql("Select * from pools",engine)
pools_metadata_df.head()

In [ ]:
pools_metadata_df.columns

In [ ]:
len(pools_metadata_df.pool_id.unique())

In [ ]:
pool_daily_metrics_df = pd.read_sql("Select * from pool_daily_metrics",engine)
pool_daily_metrics_df.head()

# Load Backtest Forecasting

In [ ]:
df_2025 = pd.read_parquet("global_apy_full_backtest_2025.parquet")
df_2025_bis = pd.read_parquet("global_apy_full_backtest_2025_bis.parquet")
forecasts_df = pd.concat(
    [df_2025, df_2025_bis],
    axis=0,
    ignore_index=True
)

forecasts_df = (
    forecasts_df
    .sort_values(["pool_id", "target_date"])
    .drop_duplicates(
        subset=["pool_id", "target_date"],
        keep="last"   # keep the most recent version (important if _bis is a fix)
    )
    .reset_index(drop=True)
)
forecasts_df = forecasts_df.sort_values("exec_date")
forecasts_df

# Create Initial Input Data

In [ ]:
warm_wallet = {'USDC': 20_000_000}
current_allocations = {}

In [ ]:
def make_backtest_gas_fees(
    eth_price_usd: float = 3000.0,
    base_fee_transfer_gwei: float = 10.0,
    base_fee_swap_gwei: float = 30.0,
    priority_fee_gwei: float = 10.0,
    min_gas_units: int = 21000,
) -> dict:
    """
    Reproduce your production gas-fee logic for backtests.
    Returns a dict suitable for AllocationOptimizer.
    """

    def gas_fee_usd(gas_units: float, base_fee_gwei: float) -> float:
        total_fee_gwei = gas_units * (base_fee_gwei + priority_fee_gwei)
        return total_fee_gwei * 1e-9 * eth_price_usd

    pool_tx_fee = gas_fee_usd(min_gas_units, base_fee_transfer_gwei)
    swap_fee    = gas_fee_usd(min_gas_units, base_fee_swap_gwei)

    return {
        "allocation": pool_tx_fee,   # deposit to pool
        "withdrawal": pool_tx_fee,   # withdraw from pool
        "conversion": swap_fee,      # swap between tokens
        "transfer":   pool_tx_fee,   # generic transfer
        "deposit":    pool_tx_fee,   # same as allocation
    }

In [ ]:
gas_fees = make_backtest_gas_fees(eth_price_usd=3000.0)
gas_fees

In [ ]:
def build_optimizer_pools_df(
    forecasts_df: pd.DataFrame,
    date: date,
    pool_daily_metrics_df: pd.DataFrame,
    pools_metadata_df: pd.DataFrame,
    apy_col="pred_global_apy_risk_adj",
    tvl_col="pred_global_tvl_risk_adj",
):
    """
    Returns a dataframe formatted exactly like the notebook’s pools_df,
    ready to be passed into AllocationOptimizer.
    """
    # 0) Filter forecasts and pool_daily_metrics to only include the specified date
    # --- normalize date → UTC midnight, tz-safe ---
    ts = pd.to_datetime(date)
    if getattr(ts, "tz", None) is not None:
        ts = ts.tz_convert("UTC").normalize()
    else:
        ts = ts.tz_localize("UTC").normalize()

    # 0) Filter forecasts and pool_daily_metrics to only include the specified date
    forecasts_df = forecasts_df[
        pd.to_datetime(forecasts_df["exec_date"]).dt.tz_convert("UTC").dt.normalize() == ts
    ].reset_index(drop=True)

    pool_daily_metrics_df = pool_daily_metrics_df[
        pd.to_datetime(pool_daily_metrics_df["date"]).dt.normalize() == ts.tz_localize(None)
    ].reset_index(drop=True)

    # 1) Get latest metrics for each pool (to access is_filtered_out)
    metrics_latest = (
        pool_daily_metrics_df.sort_values("date")
        .drop_duplicates(subset=["pool_id"], keep="last")[["pool_id", "is_filtered_out"]]
    )

    # 2) Merge forecasts + metadata
    df = forecasts_df.merge(
        pools_metadata_df[[
            "pool_id", "symbol", "chain", "protocol",
            "underlying_tokens", "is_active"
        ]],
        on="pool_id",
        how="left"
    )

    # 3) Add is_filtered_out from daily metrics
    df = df.merge(metrics_latest, on="pool_id", how="left")

    # 4) Rename predictions → optimizer expected fields
    df = df.rename(columns={
        apy_col: "forecasted_apy",
        tvl_col: "forecasted_tvl"
    })

    # 5) Ensure correct dtype for underlying tokens
    # (Convert string JSON to Python list)
    def parse_tokens(x):
        if isinstance(x, list):
            return x
        try:
            return json.loads(x)
        except:
            return []

    df["underlying_tokens"] = df["underlying_tokens"].apply(parse_tokens)

    # 6) Select only the required columns
    df = df[[
        "pool_id", "symbol", "chain", "protocol",
        "underlying_tokens", "forecasted_apy", "forecasted_tvl",
        "is_filtered_out", "is_active"
    ]]

    # 7) Filter out pools where is_filtered_out = True
    df = df[df["is_filtered_out"] == False].reset_index(drop=True)

    df["underlying_tokens"] = df["underlying_tokens"].apply(
    lambda x: json.dumps(x) if isinstance(x, list) else x
)

    return df

In [ ]:
# pools_df = build_optimizer_pools_df(
#     forecasts_df=forecasts_df,
#     date = "2025-11-07",
#     pool_daily_metrics_df = pool_daily_metrics_df,
#     pools_metadata_df=pools_metadata_df,
#     apy_col="pred_global_apy_risk_adj",
#     tvl_col="pred_global_tvl_risk_adj"
# )

# pools_df

In [ ]:
# Extract all underlying tokens from metadata
allowed_tokens = sorted({
    token
    for raw in pools_metadata_df["underlying_tokens"].dropna().tolist()
    for token in (
        json.loads(raw) if isinstance(raw, str)
        else (raw if isinstance(raw, list) else [])
    )
})

# Assume all stablecoins ≈ 1 USD for backtest
token_prices = {t: 1.0 for t in allowed_tokens}

token_prices

In [ ]:

print("\nInitializing optimizer...")
optimizer = AllocationOptimizer(
    pools_df=pools_df,
    token_prices=token_prices,
    warm_wallet=warm_wallet,
    current_allocations=current_allocations,
    gas_fees=gas_fees,
    alloc_params=alloc_params
)

print("✓ Optimizer initialized successfully")

In [ ]:
# Solve optimization problem
print("Solving optimization problem...")
start_time = time.time()

try:
    success = optimizer.solve(verbose=True)
    solve_time = time.time() - start_time
    
    if success:
        print(f"✓ Optimization solved successfully in {solve_time:.3f} seconds")
    else:
        print("✗ Optimization failed")
        
except Exception as e:
    print(f"✗ Error during optimization: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
if success:
    # Extract and format results
    formatted_results = optimizer.format_results()
    
    print("=" * 80)
    print("OPTIMIZATION RESULTS")
    print("=" * 80)
    
    # Display final allocations
    print("\nFINAL ALLOCATIONS:")
    total_allocated = 0
    for pool_id, pool_data in formatted_results["final_allocations"].items():
        pool_total_usd = sum(token_data['amount_usd'] for token_data in pool_data["tokens"].values())
        pool_percentage = pool_total_usd / optimizer.total_aum
        total_allocated += pool_total_usd
        print(f"\nPool: {pool_id} ({pool_data['pool_symbol']}) - Total: ${pool_total_usd:,.2f} ({pool_percentage:.2%})")
        for token, token_data in pool_data["tokens"].items():
            print(f"  {token}: {token_data['amount']:,.2f} (${token_data['amount_usd']:,.2f})")
    
    # Display unallocated tokens
    print("\nUNALLOCATED TOKENS (in warm wallet):")
    total_unallocated = 0
    for token, token_data in formatted_results["unallocated_tokens"].items():
        total_unallocated += token_data['amount_usd']
        print(f"  {token}: {token_data['amount']:,.2f} (${token_data['amount_usd']:,.2f})")
    
    # Display transaction summary
    print("\nTRANSACTION SUMMARY:")
    transactions = formatted_results["transactions"]
    total_gas_cost = sum(txn.get('gas_cost_usd', 0) for txn in transactions)
    total_conversion_cost = sum(txn.get('conversion_cost_usd', 0) for txn in transactions)
    total_cost = sum(txn.get('total_cost_usd', 0) for txn in transactions)
    
    print(f"  Total transactions: {len(transactions)}")
    print(f"  Total gas cost: ${total_gas_cost:.4f}")
    print(f"  Total conversion cost: ${total_conversion_cost:.4f}")
    print(f"  Total transaction cost: ${total_cost:.4f}")
    
    # Display allocation summary
    print("\nALLOCATION SUMMARY:")
    print(f"  Total AUM: ${optimizer.total_aum:,.2f}")
    print(f"  Total allocated: ${total_allocated:,.2f} ({total_allocated/optimizer.total_aum:.2%})")
    print(f"  Total unallocated: ${total_unallocated:,.2f} ({total_unallocated/optimizer.total_aum:.2%})")
    print(f"  Transaction costs: ${total_cost:.4f} ({total_cost/optimizer.total_aum:.4%})")
    print(f"  Net allocated: ${total_allocated + total_unallocated + total_cost:,.2f}")
    
else:
    print("No results to display - optimization failed")

In [ ]:
if success:
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Optimization Results Analysis', fontsize=16, fontweight='bold')
    
    # 1. Allocation by pool
    pool_data = []
    for pool_id, pool_info in formatted_results["final_allocations"].items():
        pool_total = sum(token_data['amount_usd'] for token_data in pool_info["tokens"].values())
        pool_data.append({
            'pool': f"{pool_id}\n({pool_info['pool_symbol']})",
            'amount_usd': pool_total,
            'percentage': pool_total / optimizer.total_aum * 100
        })
    
    pool_df = pd.DataFrame(pool_data)
    pool_df = pool_df.sort_values('amount_usd', ascending=False)
    
    axes[0, 0].bar(range(len(pool_df)), pool_df['amount_usd'])
    axes[0, 0].set_title('Allocation by Pool')
    axes[0, 0].set_xlabel('Pool')
    axes[0, 0].set_ylabel('Amount (USD)')
    axes[0, 0].set_xticks(range(len(pool_df)))
    axes[0, 0].set_xticklabels(pool_df['pool'], rotation=45, ha='right')
    
    # Add percentage labels on bars
    for i, row in pool_df.iterrows():
        axes[0, 0].text(i, row['amount_usd'], f"{row['percentage']:.1f}%", 
                       ha='center', va='bottom')
    
    # 2. Token distribution
    token_dist = {}
    for pool_id, pool_info in formatted_results["final_allocations"].items():
        for token, token_data in pool_info["tokens"].items():
            if token not in token_dist:
                token_dist[token] = 0
            token_dist[token] += token_data['amount_usd']
    
    # Add unallocated tokens
    for token, token_data in formatted_results["unallocated_tokens"].items():
        if token not in token_dist:
            token_dist[token] = 0
        token_dist[token] += token_data['amount_usd']
    
    tokens_df = pd.DataFrame(list(token_dist.items()), columns=['token', 'amount_usd'])
    tokens_df = tokens_df.sort_values('amount_usd', ascending=False)
    
    axes[0, 1].pie(tokens_df['amount_usd'], labels=tokens_df['token'], autopct='%1.1f%%')
    axes[0, 1].set_title('Token Distribution')
    
    # 3. APY distribution
    apy_data = []
    for pool_id, pool_info in formatted_results["final_allocations"].items():
        pool_total = sum(token_data['amount_usd'] for token_data in pool_info["tokens"].values())
        pool_row = pools_df[pools_df['pool_id'] == pool_id].iloc[0]
        apy_data.append({
            'pool': pool_info['pool_symbol'],
            'apy': pool_row['forecasted_apy'],
            'amount_usd': pool_total
        })
    
    apy_df = pd.DataFrame(apy_data)
    
    axes[1, 0].scatter(apy_df['apy'], apy_df['amount_usd'], s=100, alpha=0.7)
    axes[1, 0].set_title('Allocation vs APY')
    axes[1, 0].set_xlabel('APY (%)')
    axes[1, 0].set_ylabel('Amount (USD)')
    
    # Add pool labels
    for _, row in apy_df.iterrows():
        axes[1, 0].annotate(row['pool'], (row['apy'], row['amount_usd']), 
                           xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    # 4. Transaction cost breakdown
    txn_types = ['WITHDRAWAL', 'CONVERSION', 'ALLOCATION']
    txn_costs = []
    
    for txn_type in txn_types:
        type_txns = [t for t in transactions if t['type'] == txn_type]
        total_cost = sum(t.get('total_cost_usd', 0) for t in type_txns)
        txn_costs.append(total_cost)
    
    axes[1, 1].bar(txn_types, txn_costs)
    axes[1, 1].set_title('Transaction Costs by Type')
    axes[1, 1].set_xlabel('Transaction Type')
    axes[1, 1].set_ylabel('Cost (USD)')
    
    # Add cost labels on bars
    for i, cost in enumerate(txn_costs):
        axes[1, 1].text(i, cost, f"${cost:.4f}", ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("No visualizations available - optimization failed")

In [ ]:
# Run all scenarios
scenario_results = []

for scenario in scenarios:
    result = run_scenario(
        scenario['name'],
        scenario['overrides'],
        scenario['pool_params'],
        pools_df  # Pass the existing pools_df for scenarios that don't generate new pools
    )
    scenario_results.append(result)
    
    if result['success']:
        print(f"✓ {scenario['name']} completed successfully")
        print(f"  Solve time: {result['solve_time']:.3f}s")
        print(f"  Allocation: {result['allocation_pct']:.2%}")
        print(f"  Weighted APY: {result['weighted_apy']:.2f}%")
        print(f"  Cost: {result['cost_pct']:.4%}")
    else:
        print(f"✗ {scenario['name']} failed: {result.get('error', 'Unknown error')}")

In [ ]:
# Compare scenarios
successful_results = [r for r in scenario_results if r['success']]

if successful_results:
    comparison_df = pd.DataFrame(successful_results)
    
    # Create comparison visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Scenario Comparison', fontsize=16, fontweight='bold')
    
    # 1. Allocation percentage
    axes[0, 0].bar(comparison_df['scenario'], comparison_df['allocation_pct'])
    axes[0, 0].set_title('Allocation Percentage')
    axes[0, 0].set_ylabel('Percentage of AUM')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Add percentage labels
    for i, pct in enumerate(comparison_df['allocation_pct']):
        axes[0, 0].text(i, pct, f"{pct:.1%}", ha='center', va='bottom')
    
    # 2. Weighted APY
    axes[0, 1].bar(comparison_df['scenario'], comparison_df['weighted_apy'])
    axes[0, 1].set_title('Weighted Average APY')
    axes[0, 1].set_ylabel('APY (%)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Add APY labels
    for i, apy in enumerate(comparison_df['weighted_apy']):
        axes[0, 1].text(i, apy, f"{apy:.2f}%", ha='center', va='bottom')
    
    # 3. Cost percentage
    axes[1, 0].bar(comparison_df['scenario'], comparison_df['cost_pct'] * 100)
    axes[1, 0].set_title('Transaction Cost Percentage')
    axes[1, 0].set_ylabel('Cost (% of AUM)')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Add cost labels
    for i, cost in enumerate(comparison_df['cost_pct']):
        axes[1, 0].text(i, cost * 100, f"{cost*100:.3f}%", ha='center', va='bottom')
    
    # 4. Solve time
    axes[1, 1].bar(comparison_df['scenario'], comparison_df['solve_time'])
    axes[1, 1].set_title('Solve Time')
    axes[1, 1].set_ylabel('Time (seconds)')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    # Add time labels
    for i, time_val in enumerate(comparison_df['solve_time']):
        axes[1, 1].text(i, time_val, f"{time_val:.3f}s", ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Display comparison table
    print("\nSCENARIO COMPARISON TABLE:")
    display_cols = ['scenario', 'allocation_pct', 'weighted_apy', 'cost_pct', 'solve_time', 'num_pools']
    display_df = comparison_df[display_cols].copy()
    display_df['allocation_pct'] = display_df['allocation_pct'].apply(lambda x: f"{x:.2%}")
    display_df['cost_pct'] = display_df['cost_pct'].apply(lambda x: f"{x:.4%}")
    display_df['weighted_apy'] = display_df['weighted_apy'].apply(lambda x: f"{x:.2f}%")
    display_df['solve_time'] = display_df['solve_time'].apply(lambda x: f"{x:.3f}s")
    
    display(display_df)
    
else:
    print("No successful scenarios to compare")

In [ ]:
import pandas as pd
import numpy as np

def extract_state_from_results(formatted_results):
    """
    Convert AllocationOptimizer.format_results() → (warm_wallet, current_allocations)
    warm_wallet: {token: amount}
    current_allocations: {(pool_id, token): amount}
    """
    warm_wallet_next = {
        token: data["amount"]
        for token, data in formatted_results["unallocated_tokens"].items()
    }

    current_allocations_next = {}
    for pool_id, pool_data in formatted_results["final_allocations"].items():
        for token, token_data in pool_data["tokens"].items():
            key = (pool_id, token)
            current_allocations_next[key] = (
                current_allocations_next.get(key, 0.0) + float(token_data["amount"])
            )

    return warm_wallet_next, current_allocations_next


def apply_daily_yield_and_pnl(
    allocations_after_opt: dict,
    pools_df: pd.DataFrame,
    pool_daily_metrics_df: pd.DataFrame,
    target_date: pd.Timestamp,
    token_prices: dict,
):
    """
    Given allocations right AFTER optimization (day t),
    apply one day of yield (t → t+1) using:
      - forecasted APY from pools_df (decision APY)
      - actual APY from pool_daily_metrics_df on target_date (realized APY)

    Returns:
      next_allocations  – dict[(pool_id, token)] = amount AFTER yield
      daily_actual_pnl  – float in USD
      daily_forecast_pnl – float in USD
    """
    # APY used for decisions (same as optimizer input)
    forecast_apy_by_pool = (
        pools_df.set_index("pool_id")["forecasted_apy"].to_dict()
        if "forecasted_apy" in pools_df.columns
        else {}
    )

    # Realized APY next day
    metrics_target = pool_daily_metrics_df[
        pool_daily_metrics_df["date"] == target_date
    ][["pool_id", "actual_apy"]]

    actual_apy_by_pool = metrics_target.set_index("pool_id")["actual_apy"].to_dict()

    next_allocations = {}
    daily_actual_pnl = 0.0
    daily_forecast_pnl = 0.0

    for (pool_id, token), amount in allocations_after_opt.items():
        price = token_prices.get(token, 1.0)

        f_apy = forecast_apy_by_pool.get(pool_id, 0.0)
        a_apy = actual_apy_by_pool.get(pool_id, f_apy)  # fallback = forecast

        f_rate = f_apy / 100.0 / 365.0
        a_rate = a_apy / 100.0 / 365.0

        value_usd = amount * price

        daily_forecast_pnl += value_usd * f_rate
        daily_actual_pnl += value_usd * a_rate

        new_amount = amount * (1.0 + a_rate)
        next_allocations[(pool_id, token)] = (
            next_allocations.get((pool_id, token), 0.0) + new_amount
        )

    return next_allocations, float(daily_actual_pnl), float(daily_forecast_pnl)


def run_optimizer_one_day_backtest(
    exec_date: pd.Timestamp,
    forecasts_df: pd.DataFrame,
    pool_daily_metrics_df: pd.DataFrame,
    pools_metadata_df: pd.DataFrame,
    token_prices: dict,
    alloc_params: dict,
    warm_wallet: dict,
    current_allocations: dict,
    apy_col: str = "pred_global_apy_risk_adj",
    tvl_col: str = "pred_global_tvl_risk_adj",
    gas_fees: dict | None = None,
):
    exec_date = pd.to_datetime(exec_date)
    exec_date_norm = exec_date.normalize()

    day_forecasts = forecasts_df[
        pd.to_datetime(forecasts_df["exec_date"]).dt.normalize() == exec_date_norm
    ]
    if day_forecasts.empty:
        # ... same as before ...
        ...

    target_date = pd.to_datetime(day_forecasts["target_date"].iloc[0]).normalize()

    # NOTE: pass pure date object to avoid tz headaches inside build_optimizer_pools_df
    pools_df = build_optimizer_pools_df(
        forecasts_df=forecasts_df,
        date=exec_date_norm.date(),      # <--- change here
        pool_daily_metrics_df=pool_daily_metrics_df,
        pools_metadata_df=pools_metadata_df,
        apy_col=apy_col,
        tvl_col=tvl_col,
    )

    # Filter out pools that are marked is_filtered_out = True (as you said)
    if "is_filtered_out" in pools_df.columns:
        pools_df = pools_df[~pools_df["is_filtered_out"].fillna(False)]

    aum_start = calculate_aum(warm_wallet, current_allocations, token_prices)

    # If no candidate pools, just apply yield on existing positions and move on
    if pools_df.empty:
        # apply yield using actual APY on target_date + forecast from previous day if you want
        allocations_after_opt = current_allocations.copy()
        next_allocs, daily_actual_pnl, daily_forecast_pnl = apply_daily_yield_and_pnl(
            allocations_after_opt,
            pools_df=pd.DataFrame(columns=["pool_id", "forecasted_apy"]),  # empty
            pool_daily_metrics_df=pool_daily_metrics_df,
            target_date=target_date,
            token_prices=token_prices,
        )

        warm_next = warm_wallet.copy()
        aum_end = calculate_aum(warm_next, next_allocs, token_prices)

        return {
            "status": "no_pools",
            "exec_date": exec_date,
            "target_date": target_date,
            "warm_wallet_next": warm_next,
            "allocations_next": next_allocs,
            "aum_start": aum_start,
            "aum_after_opt": aum_start,
            "aum_end": aum_end,
            "daily_actual_pnl": daily_actual_pnl,
            "daily_forecast_pnl": daily_forecast_pnl,
            "total_fees_usd": 0.0,
            "n_alloc_positions": len(next_allocs),
        }

    # --- gas fees for this day ---
    if gas_fees is None:
        # constant fees version (you already use it)
        gas_fees = make_backtest_gas_fees(eth_price_usd=3000.0)

    # --- run optimizer (this part is basically your one-day code, but without plots/logging) ---
    optimizer = AllocationOptimizer(
        pools_df=pools_df,
        token_prices=token_prices,
        warm_wallet=warm_wallet,
        current_allocations=current_allocations,
        gas_fees=gas_fees,
        alloc_params=alloc_params,
    )

    success = optimizer.solve(verbose=False)

    if not success:
        # If solver fails, keep portfolio as is and just apply yield
        allocations_after_opt = current_allocations.copy()
        warm_after_opt = warm_wallet.copy()
        aum_after_opt = calculate_aum(warm_after_opt, allocations_after_opt, token_prices)
        next_allocs, daily_actual_pnl, daily_forecast_pnl = apply_daily_yield_and_pnl(
            allocations_after_opt,
            pools_df,
            pool_daily_metrics_df,
            target_date,
            token_prices,
        )
        aum_end = calculate_aum(warm_after_opt, next_allocs, token_prices)

        return {
            "status": "opt_fail",
            "exec_date": exec_date,
            "target_date": target_date,
            "warm_wallet_next": warm_after_opt,
            "allocations_next": next_allocs,
            "aum_start": aum_start,
            "aum_after_opt": aum_after_opt,
            "aum_end": aum_end,
            "daily_actual_pnl": daily_actual_pnl,
            "daily_forecast_pnl": daily_forecast_pnl,
            "total_fees_usd": 0.0,
            "n_alloc_positions": len(next_allocs),
        }

    # Extract optimized allocations + warm wallet
    formatted_results = optimizer.format_results()
    warm_after_opt, allocations_after_opt = extract_state_from_results(formatted_results)

    # total transaction fees for this day
    total_fees_usd = float(
        sum(txn.get("total_cost_usd", 0.0) for txn in formatted_results["transactions"])
    )

    aum_after_opt = calculate_aum(warm_after_opt, allocations_after_opt, token_prices)

    # Apply one day of realized yield
    next_allocs, daily_actual_pnl, daily_forecast_pnl = apply_daily_yield_and_pnl(
        allocations_after_opt,
        pools_df,
        pool_daily_metrics_df,
        target_date,
        token_prices,
    )

    warm_next = warm_after_opt  # assume warm wallet tokens don’t earn yield
    aum_end = calculate_aum(warm_next, next_allocs, token_prices)

    return {
        "status": "ok",
        "exec_date": exec_date,
        "target_date": target_date,
        "warm_wallet_next": warm_next,
        "allocations_next": next_allocs,
        "aum_start": aum_start,
        "aum_after_opt": aum_after_opt,
        "aum_end": aum_end,
        "daily_actual_pnl": daily_actual_pnl,
        "daily_forecast_pnl": daily_forecast_pnl,
        "total_fees_usd": total_fees_usd,
        "n_alloc_positions": len(next_allocs),
    }

In [ ]:
def collapse_allocations_by_token(pool_token_allocations: dict) -> dict:
    """
    Input:
      { (pool_id, token): usd_amount, ... }

    Output:
      { token: total_usd_amount }
    """
    out = defaultdict(float)

    for (_, token), amount in pool_token_allocations.items():
        out[token] += float(amount)

    return dict(out)

In [ ]:
from tqdm.auto import tqdm  # works nicely in notebooks

def run_optimization_backtest(
    forecasts_df: pd.DataFrame,
    pool_daily_metrics_df: pd.DataFrame,
    pools_metadata_df: pd.DataFrame,
    token_prices: dict,
    alloc_params: dict,
    initial_warm_wallet: dict,
    initial_allocations: dict | None = None,
    apy_col: str = "pred_global_apy_risk_adj",
    tvl_col: str = "pred_global_tvl_risk_adj",
) -> pd.DataFrame:
    current_warm_wallet = initial_warm_wallet.copy()
    current_allocations = (initial_allocations or {}).copy()

    forecasts_df = forecasts_df.copy()
    forecasts_df["exec_date"] = pd.to_datetime(forecasts_df["exec_date"])

    all_exec_dates = sorted(
        forecasts_df["exec_date"].dt.normalize().unique()
    )

    log_rows = []

    for d in tqdm(all_exec_dates, desc="Backtesting days"):
        res = run_optimizer_one_day_backtest(
            exec_date=d,
            forecasts_df=forecasts_df,
            pool_daily_metrics_df=pool_daily_metrics_df,
            pools_metadata_df=pools_metadata_df,
            token_prices=token_prices,
            alloc_params=alloc_params,
            warm_wallet=current_warm_wallet,
            current_allocations=current_allocations,
            apy_col=apy_col,
            tvl_col=tvl_col,
            gas_fees=make_backtest_gas_fees(eth_price_usd=3000.0),
        )
        if res["aum_start"] > 0:
            daily_apy_pct = res["daily_actual_pnl"] / res["aum_start"] * 365 * 100
        else:
            daily_apy_pct = 0.0
        tqdm.write(
            f"{pd.to_datetime(res['exec_date']).date()} | "
            f"AUM_end={res['aum_end']:,.0f} | "
            f"realized_APY≈{daily_apy_pct:.2f}% "
            f"| fees={res['total_fees_usd']:.2f}"
        )

        log_rows.append(
            {
                "exec_date": res["exec_date"],
                "target_date": res["target_date"],
                "aum_start": res["aum_start"],
                "aum_after_opt": res["aum_after_opt"],
                "aum_end": res["aum_end"],
                "daily_actual_pnl": res["daily_actual_pnl"],
                "daily_forecast_pnl": res["daily_forecast_pnl"],
                "total_fees_usd": res["total_fees_usd"],
                "n_alloc_positions": res["n_alloc_positions"],
                "allocations_by_token": collapse_allocations_by_token(res["allocations_next"]),
                "status": res["status"],
            }
        )

        current_warm_wallet = res["warm_wallet_next"]
        current_allocations = res["allocations_next"]

    backtest_log = pd.DataFrame(log_rows).sort_values("exec_date").reset_index(drop=True)
    return backtest_log

In [ ]:
import logging
import asset_allocation.optimize_allocations as opt_mod
from collections import defaultdict

# 1) Silence that specific logger
opt_logger = logging.getLogger(opt_mod.__name__)
opt_logger.setLevel(logging.ERROR)       # or logging.CRITICAL if you want complete silence
for h in opt_logger.handlers:
    h.setLevel(logging.ERROR)

# 2) Optional: silence cvxpy solver chatter as well
logging.getLogger("cvxpy").setLevel(logging.ERROR)

# 3) Optional: hide some warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# 1) Token prices
allowed_tokens = sorted({
    token
    for raw in pools_metadata_df["underlying_tokens"].dropna().tolist()
    for token in (
        json.loads(raw) if isinstance(raw, str)
        else (raw if isinstance(raw, list) else [])
    )
})
token_prices = {t: 1.0 for t in allowed_tokens}  # all stables ≈ 1 USD

# 2) Initial state
initial_warm_wallet = {"USDC": 20_000_000}
initial_allocations = {}

# 3) Run backtest
bt_results = run_optimization_backtest(
    forecasts_df=forecasts_df[:100],
    pool_daily_metrics_df=pool_daily_metrics_df,
    pools_metadata_df=pools_metadata_df,
    token_prices=token_prices,
    alloc_params=alloc_params,
    initial_warm_wallet=initial_warm_wallet,
    initial_allocations=initial_allocations,
    apy_col="pred_global_apy_risk_adj",
    tvl_col="pred_global_tvl_risk_adj",
)

bt_results.head()

In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# --- 1) APY to daily return (correct) ---------------------------------------

def apy_to_daily_return(apy_pct: float) -> float:
    """
    Convert annual APY in % (e.g. 10.0 for 10%) to daily return.

    Uses (1 + r)^(1/365) - 1, NOT r/365. Works also for negative APY.
    """
    if pd.isna(apy_pct):
        return 0.0
    r = apy_pct / 100.0
    return float((1.0 + r) ** (1.0 / 365.0) - 1.0)


# --- 2) Build a mapping of actual APY by (pool_id, date) --------------------

def build_actual_apy_lookup(pool_daily_metrics_df: pd.DataFrame) -> dict:
    """
    Returns a dict[(pool_id, date_normalized_utc) -> actual_apy_pct].
    """
    df = pool_daily_metrics_df.copy()
    df["date_norm"] = pd.to_datetime(df["date"], utc=True).dt.normalize()

    lookup = {}
    for _, row in df[["pool_id", "date_norm", "actual_apy"]].dropna(subset=["actual_apy"]).iterrows():
        lookup[(row["pool_id"], row["date_norm"])] = float(row["actual_apy"])
    return lookup


# --- 3) Compute daily PnL given allocations & APYs --------------------------

def compute_daily_pnl_from_allocations(
    allocations_usd_by_pool: dict,
    apy_by_pool: dict,
) -> float:
    """
    allocations_usd_by_pool: {pool_id: total_allocated_usd}
    apy_by_pool:            {pool_id: APY in %, annual}
    returns: total daily PnL in USD (sum over pools)
    """
    pnl = 0.0
    for pool_id, alloc_usd in allocations_usd_by_pool.items():
        apy_pct = apy_by_pool.get(pool_id, 0.0)
        dr = apy_to_daily_return(apy_pct)
        pnl += alloc_usd * dr
    return float(pnl)


# --- 4) Update token allocations with *actual* daily return -----------------

def apply_daily_yield_to_allocations(
    allocations: dict,
    actual_apy_by_pool: dict,
    token_prices: dict,
) -> dict:
    """
    allocations: {(pool_id, token): amount_tokens}
    token_prices: {token: price_usd}  (stablecoins ≈ 1.0)
    actual_apy_by_pool: {pool_id: APY%}
    
    Returns new_allocations after one day of actual yield.
    Assumes yield accrues inside the pool proportionally to capital.
    """
    new_allocations = {}

    for (pool_id, token), amount in allocations.items():
        price = token_prices.get(token, 1.0)
        apy_pct = actual_apy_by_pool.get(pool_id, 0.0)
        dr = apy_to_daily_return(apy_pct)
        growth_factor = 1.0 + dr

        # amount is in token units; for stablecoins with price≈1 this is fine
        new_amount = float(amount) * growth_factor
        new_allocations[(pool_id, token)] = new_amount

    return new_allocations

In [ ]:
def run_optimizer_one_day_backtest(
    exec_date,                    # day t (when forecast was created)
    forecasts_df: pd.DataFrame,
    pool_daily_metrics_df: pd.DataFrame,
    pools_metadata_df: pd.DataFrame,
    token_prices: dict,
    gas_fees: dict,
    alloc_params: dict,
    warm_wallet: dict,
    current_allocations: dict,
    aum_start: float,             # <<< pass scalar AUM explicitly
    apy_col: str = "pred_global_apy_risk_adj",
    tvl_col: str = "pred_global_tvl_risk_adj",
) -> dict:
    """
    One backtest step: from exec_date (t) to target_date (t+1).
    Uses AllocationOptimizer for reallocation but keeps AUM evolution explicit.
    """

    # ------------------------------------------------------------------#
    # 0) Normalize dates
    # ------------------------------------------------------------------#
    exec_ts = pd.to_datetime(exec_date)
    if exec_ts.tzinfo is None:
        exec_ts = exec_ts.tz_localize("UTC")
    else:
        exec_ts = exec_ts.tz_convert("UTC")
    exec_day = exec_ts.normalize()
    target_day = exec_day + pd.Timedelta(days=1)

    # ------------------------------------------------------------------#
    # 1) Build pools_df for this day (forecasts for t, metrics for t+1)
    # ------------------------------------------------------------------#
    pools_df = build_optimizer_pools_df(
        forecasts_df=forecasts_df,
        date=exec_day,
        pool_daily_metrics_df=pool_daily_metrics_df,
        pools_metadata_df=pools_metadata_df,
        apy_col=apy_col,
        tvl_col=tvl_col,
    )

    # Build actual APY lookup for use later
    actual_lookup = build_actual_apy_lookup(pool_daily_metrics_df)

    # Helper: compute allocations_usd_by_pool from current_allocations
    def alloc_usd_by_pool_from_state(allocs: dict) -> dict:
        m = {}
        for (pool_id, token), amount in allocs.items():
            price = token_prices.get(token, 1.0)
            m[pool_id] = m.get(pool_id, 0.0) + amount * price
        return m

    # ------------------------------------------------------------------#
    # SPECIAL CASE: no eligible pools or optimizer can’t run
    # In this case we **don’t** reallocate, only apply yield.
    # ------------------------------------------------------------------#
    if pools_df.empty:
        allocations_usd_by_pool = alloc_usd_by_pool_from_state(current_allocations)

        actual_apy_by_pool = {
            pid: actual_lookup.get((pid, target_day), 0.0)
            for pid in allocations_usd_by_pool.keys()
        }

        # PnL from actual APY
        daily_actual_pnl = compute_daily_pnl_from_allocations(
            allocations_usd_by_pool, actual_apy_by_pool
        )
        daily_forecast_pnl = daily_actual_pnl  # no forecasts in this branch
        daily_fees_usd = 0.0

        # Apply yield to positions
        next_allocations = apply_daily_yield_to_allocations(
            current_allocations, actual_apy_by_pool, token_prices
        )
        next_warm_wallet = dict(warm_wallet)

        aum_after_opt = aum_start          # no fees, no reallocation
        aum_end = aum_after_opt + daily_actual_pnl

        realized_daily_ret = daily_actual_pnl / aum_after_opt if aum_after_opt > 0 else 0.0
        realized_apy_pct = realized_daily_ret * 365.0 * 100.0

        return dict(
            exec_date=exec_day,
            target_date=target_day,
            status="no_pools",
            aum_start=aum_start,
            aum_after_opt=aum_after_opt,
            aum_end=aum_end,
            daily_actual_pnl=daily_actual_pnl,
            daily_forecast_pnl=daily_forecast_pnl,
            daily_fees_usd=daily_fees_usd,
            realized_apy_pct=realized_apy_pct,
            n_alloc_positions=len(next_allocations),
            next_warm_wallet=next_warm_wallet,
            next_allocations=next_allocations,
            allocations_by_token = collapse_allocations_by_token(next_allocations)
        )

    # ------------------------------------------------------------------#
    # 2) Run optimizer
    # ------------------------------------------------------------------#
    optimizer = AllocationOptimizer(
        pools_df=pools_df,
        token_prices=token_prices,
        warm_wallet=warm_wallet,
        current_allocations=current_allocations,
        gas_fees=gas_fees,
        alloc_params=alloc_params,
    )

    success = optimizer.solve(verbose=False)

    if not success:
        # Fall back to "hold & earn yield" behaviour
        allocations_usd_by_pool = alloc_usd_by_pool_from_state(current_allocations)

        actual_apy_by_pool = {
            pid: actual_lookup.get((pid, target_day), 0.0)
            for pid in allocations_usd_by_pool.keys()
        }

        daily_actual_pnl = compute_daily_pnl_from_allocations(
            allocations_usd_by_pool, actual_apy_by_pool
        )
        daily_forecast_pnl = daily_actual_pnl
        daily_fees_usd = 0.0

        next_allocations = apply_daily_yield_to_allocations(
            current_allocations, actual_apy_by_pool, token_prices
        )
        next_warm_wallet = dict(warm_wallet)

        aum_after_opt = aum_start
        aum_end = aum_after_opt + daily_actual_pnl

        realized_daily_ret = daily_actual_pnl / aum_after_opt if aum_after_opt > 0 else 0.0
        realized_apy_pct = realized_daily_ret * 365.0 * 100.0

        return dict(
            exec_date=exec_day,
            target_date=target_day,
            status="opt_failed",
            aum_start=aum_start,
            aum_after_opt=aum_after_opt,
            aum_end=aum_end,
            daily_actual_pnl=daily_actual_pnl,
            daily_forecast_pnl=daily_forecast_pnl,
            daily_fees_usd=daily_fees_usd,
            realized_apy_pct=realized_apy_pct,
            n_alloc_positions=len(next_allocations),
            next_warm_wallet=next_warm_wallet,
            next_allocations=next_allocations,
            allocations_by_token = collapse_allocations_by_token(next_allocations)
        )

    # ------------------------------------------------------------------#
    # 3) Extract results from optimizer (allocations + fees)
    # ------------------------------------------------------------------#
    formatted = optimizer.format_results()

    # New warm wallet balances (after paying fees & reallocating)
    warm_after = {
        token: data["amount"]
        for token, data in formatted["unallocated_tokens"].items()
    }

    # New allocations per (pool, token)
    alloc_after = {}
    allocations_usd_by_pool = {}
    for pool_id, pool_data in formatted["final_allocations"].items():
        pool_total_usd = 0.0
        for token, tok_data in pool_data["tokens"].items():
            key = (pool_id, token)
            alloc_after[key] = float(tok_data["amount"])
            pool_total_usd += float(tok_data["amount_usd"])
        allocations_usd_by_pool[pool_id] = allocations_usd_by_pool.get(pool_id, 0.0) + pool_total_usd

    # Transaction fees for logging
    daily_fees_usd = float(
        sum(txn.get("total_cost_usd", 0.0) for txn in formatted["transactions"])
    )

    # >>> HERE we enforce the accounting:
    # AUM_after_opt = AUM_start - fees   (reallocation itself should not destroy value)
    aum_after_opt = aum_start - daily_fees_usd

    # (Optional sanity check – you can keep or comment out)
    # token_based_aum = (
    #     sum(allocations_usd_by_pool.values()) +
    #     sum(warm_after[t] * token_prices.get(t, 1.0) for t in warm_after)
    # )
    # if abs(token_based_aum - aum_after_opt) > 1e-3:
    #     print("WARNING: token-based AUM mismatch:", token_based_aum, "vs", aum_after_opt)

    # ------------------------------------------------------------------#
    # 4) Daily PnL: forecast vs actual APY on new allocations
    # ------------------------------------------------------------------#
    # Forecast APY per pool
    forecast_apy_by_pool = {
        row["pool_id"]: float(row["forecasted_apy"])
        for _, row in pools_df[["pool_id", "forecasted_apy"]].iterrows()
    }

    # Actual APY per pool for target_day
    actual_apy_by_pool = {
        pid: actual_lookup.get((pid, target_day), 0.0)
        for pid in allocations_usd_by_pool.keys()
    }

    daily_forecast_pnl = compute_daily_pnl_from_allocations(
        allocations_usd_by_pool, forecast_apy_by_pool
    )
    daily_actual_pnl = compute_daily_pnl_from_allocations(
        allocations_usd_by_pool, actual_apy_by_pool
    )

    # ------------------------------------------------------------------#
    # 5) Apply actual yield to get next day's positions
    # ------------------------------------------------------------------#
    next_allocations = apply_daily_yield_to_allocations(
        alloc_after, actual_apy_by_pool, token_prices
    )
    next_warm_wallet = dict(warm_after)

    aum_end = aum_after_opt + daily_actual_pnl

    realized_daily_ret = daily_actual_pnl / aum_after_opt if aum_after_opt > 0 else 0.0
    realized_apy_pct = realized_daily_ret * 365.0 * 100.0

    return dict(
        exec_date=exec_day,
        target_date=target_day,
        status="ok",
        aum_start=aum_start,
        aum_after_opt=aum_after_opt,
        aum_end=aum_end,
        daily_actual_pnl=daily_actual_pnl,
        daily_forecast_pnl=daily_forecast_pnl,
        daily_fees_usd=daily_fees_usd,
        realized_apy_pct=realized_apy_pct,
        n_alloc_positions=len(next_allocations),
        next_warm_wallet=next_warm_wallet,
        next_allocations=next_allocations,
        allocations_by_token = collapse_allocations_by_token(next_allocations)
    )

In [ ]:
def run_optimization_backtest(
    forecasts_df: pd.DataFrame,
    pool_daily_metrics_df: pd.DataFrame,
    pools_metadata_df: pd.DataFrame,
    token_prices: dict,
    alloc_params: dict,
    initial_warm_wallet: dict,
    initial_allocations: dict,
    apy_col: str = "pred_global_apy_risk_adj",
    tvl_col: str = "pred_global_tvl_risk_adj",
    eth_price_usd: float = 3000.0,
) -> pd.DataFrame:

    fdf = forecasts_df.copy()
    fdf["exec_date_norm"] = pd.to_datetime(fdf["exec_date"], utc=True).dt.normalize()
    all_exec_dates = sorted(fdf["exec_date_norm"].unique())

    warm_wallet = initial_warm_wallet.copy()
    allocations = initial_allocations.copy()

    # AUM is scalar, initialise from current state or from initial capital
    aum = calculate_aum(warm_wallet, allocations, token_prices)

    log_rows = []

    for d in tqdm(all_exec_dates, desc="Backtesting days"):
        gas_fees = make_backtest_gas_fees(eth_price_usd=eth_price_usd)

        res = run_optimizer_one_day_backtest(
            exec_date=d,
            forecasts_df=forecasts_df,
            pool_daily_metrics_df=pool_daily_metrics_df,
            pools_metadata_df=pools_metadata_df,
            token_prices=token_prices,
            gas_fees=gas_fees,
            alloc_params=alloc_params,
            warm_wallet=warm_wallet,
            current_allocations=allocations,
            aum_start=aum,          # <<< pass scalar AUM
            apy_col=apy_col,
            tvl_col=tvl_col,
        )

        log_rows.append(
            dict(
                exec_date=res["exec_date"],
                target_date=res["target_date"],
                status=res["status"],
                aum_start=res["aum_start"],
                aum_after_opt=res["aum_after_opt"],
                aum_end=res["aum_end"],
                daily_actual_pnl=res["daily_actual_pnl"],
                daily_forecast_pnl=res["daily_forecast_pnl"],
                daily_fees_usd=res["daily_fees_usd"],
                realized_apy_pct=res["realized_apy_pct"],
                n_alloc_positions=res["n_alloc_positions"],
                allocations_by_token = collapse_allocations_by_token(res["next_allocations"])

                
            )
        )

        # nice compact line per day
        tqdm.write(
            f"{res['exec_date'].date()} | "
            f"AUM_end={res['aum_end']:,.0f} | "
            f"realized_APY≈{res['realized_apy_pct']:.2f}% | "
            f"fees={res['daily_fees_usd']:.2f} | status={res['status']}"
        )

        # update state for next day
        warm_wallet = res["next_warm_wallet"]
        allocations = res["next_allocations"]
        aum = res["aum_end"]      # <<< core recursion

    bt_results = pd.DataFrame(log_rows)
    return bt_results

In [ ]:
# token universe from metadata
allowed_tokens = sorted({
    token
    for raw in pools_metadata_df["underlying_tokens"].dropna().tolist()
    for token in (
        json.loads(raw) if isinstance(raw, str)
        else (raw if isinstance(raw, list) else [])
    )
})
token_prices = {t: 1.0 for t in allowed_tokens}  # stablecoins ~ 1 USD

initial_warm_wallet = {"USDC": 20_000.0}
initial_allocations = {}  # start fully in cash

bt_results = run_optimization_backtest(
    forecasts_df=forecasts_df[:100],
    pool_daily_metrics_df=pool_daily_metrics_df,
    pools_metadata_df=pools_metadata_df,
    token_prices=token_prices,
    alloc_params=alloc_params,
    initial_warm_wallet=initial_warm_wallet,
    initial_allocations=initial_allocations,
    apy_col="pred_global_apy_risk_adj",
    tvl_col="pred_global_tvl_risk_adj",
    eth_price_usd=3000.0,  # or a series later
)

bt_results.head()

In [ ]:
from collections import defaultdict

def collapse_pool_allocations_to_tokens(
    pool_allocations: dict,
    pools_metadata_df: pd.DataFrame,
) -> dict:
    """
    pool_allocations:
        { pool_id: usd_amount }

    pools_metadata_df:
        must contain columns ['pool_id', 'underlying_tokens']

    returns:
        { token: total_usd_amount }
    """
    # build fast lookup: pool_id -> [tokens]
    pool_to_tokens = {
        str(r["pool_id"]): r["underlying_tokens"]
        for _, r in pools_metadata_df.iterrows()
        if isinstance(r.get("underlying_tokens"), (list, tuple))
    }

    out = defaultdict(float)

    for pool_id, amount in pool_allocations.items():
        tokens = pool_to_tokens.get(str(pool_id))
        if not tokens:
            continue

        split = float(amount) / len(tokens)
        for t in tokens:
            out[t] += split

    return dict(out)

In [ ]:
def build_optimizer_pools_df(
    forecasts_df: pd.DataFrame,
    exec_date,
    pool_daily_metrics_df: pd.DataFrame,
    pools_metadata_df: pd.DataFrame,
    apy_col: str = "pred_global_apy_risk_adj",
    tvl_col: str = "pred_global_tvl_risk_adj",
) -> pd.DataFrame:
    exec_day = _as_utc_day(exec_date)

    # --- forecasts for that exec day ---
    f = forecasts_df.copy()
    f["exec_day"] = pd.to_datetime(f["exec_date"], utc=True).dt.normalize()
    f = f[f["exec_day"] == exec_day].copy()

    # keep only needed columns
    f = f[["pool_id", apy_col, tvl_col]].rename(
        columns={apy_col: "forecasted_apy", tvl_col: "forecasted_tvl"}
    )

    # --- daily metrics for that calendar date ---
    m = pool_daily_metrics_df.copy()
    m["day"] = pd.to_datetime(m["date"], utc=True, errors="coerce").dt.normalize()
    m = m[m["day"] == exec_day].copy()

    # keep filter flag + actuals if you want later
    m = m[["pool_id", "is_filtered_out", "actual_apy", "actual_tvl", "forecasted_apy", "forecasted_tvl"]]

    # --- metadata ---
    meta = pools_metadata_df.copy()
    meta["underlying_tokens"] = meta["underlying_tokens"].apply(_safe_json_load)

    meta = meta[["pool_id", "symbol", "chain", "protocol", "underlying_tokens", "is_active"]]

    # --- merge ---
    df = (
        f.merge(meta, on="pool_id", how="left")
         .merge(m, on="pool_id", how="left", suffixes=("", "_metrics"))
    )

    # IMPORTANT: filter based on daily metrics is_filtered_out == False
    df["is_filtered_out"] = df["is_filtered_out"].fillna(True)
    df = df[df["is_filtered_out"] == False].copy()

    # basic sanity
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna(subset=["forecasted_apy", "forecasted_tvl"])

    # ensure list type for underlying_tokens
    df["underlying_tokens"] = df["underlying_tokens"].apply(lambda x: x if isinstance(x, list) else [])

    # keep columns expected by optimizer
    out = df[[
        "pool_id", "symbol", "chain", "protocol", "underlying_tokens",
        "forecasted_apy", "forecasted_tvl", "is_filtered_out", "is_active",
        "actual_apy", "actual_tvl"
    ]].copy()

    return out

In [ ]:
def run_independent_daily_backtest(
    forecasts_df: pd.DataFrame,
    pool_daily_metrics_df: pd.DataFrame,
    pools_metadata_df: pd.DataFrame,
    token_prices: dict,
    gas_fees: dict,
    alloc_params: dict,
    fixed_aum: float = 20_000_000.0,
    apy_col: str = "pred_global_apy_risk_adj",
    tvl_col: str = "pred_global_tvl_risk_adj",
    verbose: bool = True,
) -> pd.DataFrame:
    """
    Backtest where **each day is optimized independently**
    with the same fixed AUM and no position carry-over.
    Logs APY + fees for every iteration.
    """

    # unique exec_dates sorted
    exec_dates = (
        pd.to_datetime(forecasts_df["exec_date"], utc=True)
        .dt.normalize()
        .drop_duplicates()
        .sort_values()
        .tolist()
    )

    rows = []
    pbar = tqdm(exec_dates, desc="Independent daily backtest")

    for d in pbar:
        res = run_one_day_fixed_aum(
            exec_date=d,
            forecasts_df=forecasts_df,
            pool_daily_metrics_df=pool_daily_metrics_df,
            pools_metadata_df=pools_metadata_df,
            token_prices=token_prices,
            gas_fees=gas_fees,
            alloc_params=alloc_params,
            fixed_aum=fixed_aum,
            apy_col=apy_col,
            tvl_col=tvl_col,
        )
        rows.append(res)

        # ---- LOGGING (clean, 1 line per day) ----
        if verbose:
            pbar.write(
                f"{res['exec_date'].date()} | "
                f"alloc=${res['total_allocated_usd']:,.0f} | "
                f"realized_APY={res['realized_apy_pct']:.2f}% | "
                f"fees=${res['daily_fees_usd']:.2f} | "
                f"status={res['status']}"
            )

    return pd.DataFrame(rows)

In [ ]:
alloc_params['min_pools'] = 5
alloc_params['group1_max_pct'] = 1
alloc_params['group2_max_pct'] = 1
alloc_params['group3_max_pct'] = 1

In [ ]:
fixed_aum = 20_000_000.0  # for example

bt_independent = run_independent_daily_backtest(
    forecasts_df=forecasts_df[:100],
    pool_daily_metrics_df=pool_daily_metrics_df,
    pools_metadata_df=pools_metadata_df,
    token_prices=token_prices,
    gas_fees=gas_fees,
    alloc_params=alloc_params,
    fixed_aum=fixed_aum,
    apy_col="pred_global_apy_risk_adj",
    tvl_col="pred_global_tvl_risk_adj",
)

bt_independent.head()

In [ ]:
fixed_aum = 20_000_000.0  # for example

bt_independent_glob = run_independent_daily_backtest(
    forecasts_df=forecasts_df[:100],
    pool_daily_metrics_df=pool_daily_metrics_df,
    pools_metadata_df=pools_metadata_df,
    token_prices=token_prices,
    gas_fees=gas_fees,
    alloc_params=alloc_params,
    fixed_aum=fixed_aum,
    apy_col="pred_global_apy",
    tvl_col="pred_global_tvl",
)

bt_independent_glob.head()

In [ ]:
def _as_utc_day(x) -> pd.Timestamp:
    """
    Normalize anything (str/date/datetime/Timestamp) to UTC midnight Timestamp.
    Safe for tz-aware inputs.
    """
    ts = pd.to_datetime(x, utc=True)
    return ts.normalize()

def _safe_json_load(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return []
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        try:
            return json.loads(x)
        except Exception:
            return []
    return []

def _to_py(x):
    if isinstance(x, (np.floating, np.integer)):
        return x.item()
    return x

def normalize_transactions(transactions):
    out = []
    for t in (transactions or []):
        tt = dict(t)
        for k, v in list(tt.items()):
            tt[k] = _to_py(v)
        out.append(tt)
    return out

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm

def run_compounded_backtest_simple(
    forecasts_df: pd.DataFrame,
    pool_daily_metrics_df: pd.DataFrame,
    pools_metadata_df: pd.DataFrame,
    token_prices: dict,
    gas_fees: dict,
    alloc_params: dict,
    initial_aum: float = 20_000_000.0,
    apy_col: str = "pred_global_apy_risk_adj",
    tvl_col: str = "pred_global_tvl_risk_adj",
    verbose: bool = True,
    # --- new ---
    ckpt_path: str | Path | None = "bt_ckpt.parquet",
    resume: bool = True,
    save_every: int = 1,  # save every N days
) -> pd.DataFrame:

    exec_dates = (
        pd.to_datetime(forecasts_df["exec_date"], utc=True)
        .dt.normalize()
        .drop_duplicates()
        .sort_values()
        .tolist()
    )

    ckpt_path = Path(ckpt_path) if ckpt_path is not None else None

    # -----------------------------
    # Resume: load existing logs + last aum_end
    # -----------------------------
    log_rows: list[dict] = []
    current_aum = float(initial_aum)
    start_from = exec_dates[0] if exec_dates else None

    if resume and ckpt_path is not None and ckpt_path.exists():
        prev = pd.read_parquet(ckpt_path)

        if len(prev) > 0 and "exec_date" in prev.columns and "aum_end" in prev.columns:
            prev["exec_date"] = pd.to_datetime(prev["exec_date"], utc=True).dt.normalize()
            prev = prev.sort_values("exec_date")

            last_exec = prev["exec_date"].iloc[-1]
            last_aum_end = float(prev["aum_end"].iloc[-1])

            # keep prev rows as starting logs
            log_rows = prev.to_dict("records")
            current_aum = last_aum_end
            start_from = last_exec + pd.Timedelta(days=1)

    # Filter the remaining exec_dates to run
    if start_from is not None:
        exec_dates_to_run = [d for d in exec_dates if d >= start_from]
    else:
        exec_dates_to_run = []

    pbar = tqdm(exec_dates_to_run, desc="Compounded backtest")

    def _flush_checkpoint(rows: list[dict]):
        """Write all rows to parquet (atomic replace)."""
        if ckpt_path is None:
            return
        df_out = pd.DataFrame(rows)
        tmp = ckpt_path.with_suffix(".tmp.parquet")
        df_out.to_parquet(tmp, index=False)
        tmp.replace(ckpt_path)

    # -----------------------------
    # Main loop
    # -----------------------------
    for i, d in enumerate(pbar, start=1):
        day_res = run_one_day_fixed_aum(
            exec_date=d,
            forecasts_df=forecasts_df,
            pool_daily_metrics_df=pool_daily_metrics_df,
            pools_metadata_df=pools_metadata_df,
            token_prices=token_prices,
            gas_fees=gas_fees,
            alloc_params=alloc_params,
            fixed_aum=current_aum,
            apy_col=apy_col,
            tvl_col=tvl_col,
        )

        daily_pnl = float(day_res.get("daily_actual_pnl", 0.0) or 0.0)
        daily_fees = float(day_res.get("daily_fees_usd", 0.0) or 0.0)
        status = day_res.get("status", "ok")

        aum_start = current_aum

        if status != "ok" or not np.isfinite(daily_pnl):
            aum_end = aum_start
            realized_apy = 0.0
        else:
            aum_end = aum_start + daily_pnl - daily_fees
            realized_apy = float(day_res.get("realized_apy_pct", 0.0) or 0.0)

        current_aum = float(aum_end)

        day_res["aum_start"] = float(aum_start)
        day_res["aum_end"] = float(aum_end)

        log_rows.append(day_res)

        pbar.set_postfix({
            "AUM_end": f"{aum_end:,.0f}",
            "APY%": f"{realized_apy:.2f}",
            "fees": f"{daily_fees:,.0f}",
            "token_changes": int(day_res.get("n_token_changes", 0)),
            "pos": int(day_res.get("n_positions", 0)),
        })

        if verbose:
            pbar.write(
                f"{pd.to_datetime(day_res['exec_date']).date()} | "
                f"AUM_start=${aum_start:,.0f} -> AUM_end=${aum_end:,.0f} | "
                f"realized_APY={realized_apy:.2f}% | "
                f"fees=${daily_fees:,.2f} | "
                f"token_changes={day_res.get('n_token_changes', 0)} | "
                f"pos={day_res.get('n_positions', 0)} | "
                f"status={status}"
            )

        # checkpoint
        if ckpt_path is not None and (i % save_every == 0):
            _flush_checkpoint(log_rows)

    # final write
    if ckpt_path is not None:
        _flush_checkpoint(log_rows)

    return pd.DataFrame(log_rows)

In [ ]:
alloc_params['min_pools'] = 5
alloc_params['group1_max_pct'] = 1
alloc_params['group2_max_pct'] = 1
alloc_params['group3_max_pct'] = 1



pools_metadata_df["underlying_tokens"] = pools_metadata_df["underlying_tokens"].apply(_safe_json_load)

bt_compound = run_compounded_backtest_simple(
    forecasts_df=forecasts_df,
    pool_daily_metrics_df=pool_daily_metrics_df,
    pools_metadata_df=pools_metadata_df,
    token_prices=token_prices,
    gas_fees=gas_fees,
    alloc_params=alloc_params,
    initial_aum=200_000.0,
    apy_col="pred_global_apy_risk_adj",
    tvl_col="pred_global_tvl_risk_adj",
    ckpt_path="bt_20.parquet", resume=True, save_every=1
)

bt_compound.head()